# Task 2 : Create Keywords from Content

## Load Library

In [1]:
import csv,numpy,pandas,nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk import sent_tokenize, word_tokenize, pos_tag

import math
from textblob import TextBlob as tb

## Load Data

In [2]:
#Baca data CSV dengan pandas
df = pandas.read_csv('../data/ds_asg_data.csv')

df.head()

,article_id,article_topic,article_content
0,93205794,Internasional,Kepolisian Inggris tengah memburu pelaku yang...
1,93186698,Ekonomi,Seluruh layanan transaksi di jalan tol akan m...
2,93191463,Teknologi,"\nHari ini, Rabu (23/8), ternyata menjadi har..."
3,93219292,Ekonomi,Saat ini Indonesia hanya memiliki cadangan ba...
4,343106,Hiburan,"Hari ini, Selasa (1/8), pedangdut Ridho Rhoma..."


In [3]:
#View berdasar topic
groupby= df.groupby('article_topic').size()
print (groupby)

article_topic
Bisnis             25
Bojonegoro        260
Ekonomi          1762
Haji             1497
Health            131
Hiburan          1466
Horor              50
Hukum              85
Internasional     741
Jakarta            12
K-Pop              61
KPK                37
Kesehatan         195
Keuangan           14
Lifestyle         572
MotoGP             35
Obat-obatan        58
Otomotif          174
Pendidikan         70
Personal           81
Pilgub Jatim       25
Politik           104
Regional           35
Sains             174
Sejarah            70
Sepak Bola       1184
Sports            435
Teknologi         571
Travel             76
dtype: int64


View berdasarkan topik dan didapatkan 29 topik dengan topik Ekonomi mempunyai proporsi paling banyak, dan dapat dilihat data tersebut tidak proporsional

Dicek dahulu apakah data terdapat missing value

In [4]:
#Cek data dan normalisasi
#cek bila ada yang kosong
cek_bentuk= df['article_content'].shape
cek_null=df.isnull()
cek_jumlah_null=df.isnull().sum()

print (cek_bentuk)
print (cek_null)
print (cek_jumlah_null)

(10000,)
      article_id  article_topic  article_content
0          False          False            False
1          False          False            False
2          False          False            False
3          False          False            False
4          False          False            False
5          False          False            False
6          False          False            False
7          False          False            False
8          False          False            False
9          False          False            False
10         False          False            False
11         False          False            False
12         False          False            False
13         False          False            False
14         False          False            False
15         False          False            False
16         False          False            False
17         False          False            False
18         False          False            False
19         

Setelah cek data apakah ada data yang null atau tidak, karena ada data yang null/missing value maka perlu dinormalisasi terlebih dahulu.

Kita hapus data yang terdapat missing value

In [5]:
#Drop/Hapus data yang kosong
modifiedData = df.dropna()

cek_bentuk= modifiedData['article_content'].shape
cek_null=modifiedData.isnull()
cek_jumlah_null=modifiedData.isnull().sum()

#Save Data yang sudah dimodifikasi ->opsional
modifiedData.to_csv('modifiedData.csv',index=False)

print (cek_bentuk)
print (cek_null)
print (cek_jumlah_null)

(9964,)
      article_id  article_topic  article_content
0          False          False            False
1          False          False            False
2          False          False            False
3          False          False            False
4          False          False            False
5          False          False            False
6          False          False            False
7          False          False            False
8          False          False            False
9          False          False            False
10         False          False            False
11         False          False            False
12         False          False            False
13         False          False            False
14         False          False            False
15         False          False            False
16         False          False            False
17         False          False            False
18         False          False            False
19         F

Kita Hapus data yang missing value, maka 36 data yang kosong tersebut dihilangkan. Lalu kita load lagi data yang telah dinormalisasi.

In [6]:
#Baca data CSV dengan pandas
df = pandas.read_csv('modifiedData.csv')
Id=df['article_id']
Con= df['article_content']
Topic = df['article_topic']

Setelah itu dilakukan tahap tokenisasi yaitu memecah kalimat menjadi kata-kata yang dilakukan untuk menjadikan sebuah kalimat menjadi lebih bermakna.

In [7]:
#Pisahkan kata/tokenize data

PK= [nltk.word_tokenize(PisahKata) for PisahKata in df['article_content']]

Setelah itu masuk ketahap stopword removal/filtrasi yaitu filtrasi adalah tahap mengambil kata-kata penting

In [8]:
stopword = ['ada','adalah','adanya','adapun','agak','agaknya','agar','akan','akankah',
                 'akhir','akhiri','akhirnya','aku','akulah','amat','amatlah','anda','andalah',
                 'antar','antara','antaranya','apa','apaan','apabila','apakah','apalagi','apatah',
                 'artinya','asal','asalkan','atas','atau','ataukah','ataupun','awal','awalnya','bagai',
                 'bagaikan','bagaimana','bagaimanakah','bagaimanapun','bagi','bagian','bahkan','bahwa',
                 'bahwasanya','baik','bakal','bakalan','balik','banyak','bapak','baru','bawah','beberapa',
                 'begini','beginian','beginikah','beginilah','begitu','begitukah','begitulah','begitupun',
                 'bekerja','belakang','belakangan','belum','belumlah','benar','benarkah','benarlah','berada',
                 'berakhir','berakhirlah','berakhirnya','berapa','berapakah','berapalah','berapapun','berarti',
                 'berawal','berbagai','berdatangan','beri','berikan','berikut','berikutnya','berjumlah','berkali-kali',
                 'berkata','berkehendak','berkeinginan','berkenaan','berlainan','berlalu','berlangsung','berlebihan','bermacam',
                 'bermacam-macam','bermaksud','bermula','bersama','bersama-sama','bersiap','bersiap-siap','bertanya','bertanya-tanya','berturut','berturut-turut','bertutur','berujar','berupa','besar','betul','betulkah','biasa','biasanya','bila','bilakah','bisa','bisakah','boleh','bolehkah','bolehlah','buat','bukan','bukankah','bukanlah','bukannya','bulan','bung','cara','caranya','cukup','cukupkah','cukuplah','cuma','dahulu','dalam','dan','dapat','dari','daripada','datang','dekat','demi','demikian','demikianlah','dengan','depan','di','dia','diakhiri','diakhirinya','dialah','diantara','diantaranya','diberi','diberikan','diberikannya','dibuat','dibuatnya','didapat','didatangkan','digunakan','diibaratkan','diibaratkannya','diingat','diingatkan','diinginkan','dijawab','dijelaskan','dijelaskannya','dikarenakan','dikatakan','dikatakannya','dikerjakan','diketahui','diketahuinya','dikira','dilakukan','dilalui','dilihat','dimaksud','dimaksudkan','dimaksudkannya','dimaksudnya','diminta','dimintai','dimisalkan','dimulai','dimulailah','dimulainya','dimungkinkan','dini','dipastikan','diperbuat','diperbuatnya','dipergunakan','diperkirakan','diperlihatkan','diperlukan','diperlukannya','dipersoalkan','dipertanyakan','dipunyai','diri','dirinya','disampaikan','disebut','disebutkan','disebutkannya','disini','disinilah','ditambahkan','ditandaskan','ditanya','ditanyai','ditanyakan','ditegaskan','ditujukan','ditunjuk','ditunjuki','ditunjukkan','ditunjukkannya','ditunjuknya','dituturkan','dituturkannya','diucapkan','diucapkannya','diungkapkan','dong','dua','dulu','empat','enggak','enggaknya','entah','entahlah','guna','gunakan','hal','hampir','hanya','hanyalah','hari','harus','haruslah','harusnya','hendak','hendaklah','hendaknya','hingga','ia','ialah','ibarat','ibaratkan','ibaratnya','ibu','ikut','ingat','ingat-ingat','ingin','inginkah','inginkan','ini','inikah','inilah','itu','itukah','itulah','jadi','jadilah','jadinya','jangan','jangankan','janganlah','jauh','jawab','jawaban','jawabnya','jelas','jelaskan','jelaslah','jelasnya','jika','jikalau','juga','jumlah','jumlahnya','justru','kala','kalau','kalaulah','kalaupun','kalian','kami','kamilah','kamu','kamulah','kan','kapan','kapankah','kapanpun','karena','karenanya','kasus','kata','katakan','katakanlah','katanya','ke','keadaan','kebetulan','kecil','kedua','keduanya','keinginan','kelamaan','kelihatan','kelihatannya','kelima','keluar','kembali','kemudian','kemungkinan','kemungkinannya','kenapa','kepada','kepadanya','kesampaian','keseluruhan','keseluruhannya','keterlaluan','ketika','khususnya','kini','kinilah','kira','kira-kira','kiranya','kita','kitalah','kok','kurang','lagi','lagian','lah','lain','lainnya','lalu','lama','lamanya','lanjut','lanjutnya','lebih','lewat','lima','luar','macam','maka','makanya','makin','malah','malahan','mampu','mampukah','mana','manakala','manalagi','masa','masalah','masalahnya','masih','masihkah','masing','masing-masing','mau','maupun','melainkan','melakukan','melalui','melihat','melihatnya','memang','memastikan','memberi','memberikan','membuat','memerlukan','memihak','meminta','memintakan','memisalkan','memperbuat','mempergunakan','memperkirakan','memperlihatkan','mempersiapkan','mempersoalkan','mempertanyakan','mempunyai','memulai','memungkinkan','menaiki','menambahkan','menandaskan','menanti','menanti-nanti','menantikan','menanya','menanyai','menanyakan','mendapat','mendapatkan','mendatang','mendatangi','mendatangkan','menegaskan','mengakhiri','mengapa','mengatakan','mengatakannya','mengenai','mengerjakan','mengetahui','menggunakan','menghendaki','mengibaratkan','mengibaratkannya','mengingat','mengingatkan','menginginkan','mengira','mengucapkan','mengucapkannya','mengungkapkan','menjadi','menjawab','menjelaskan','menuju','menunjuk','menunjuki','menunjukkan','menunjuknya','menurut','menuturkan','menyampaikan','menyangkut','menyatakan','menyebutkan','menyeluruh','menyiapkan','merasa','mereka','merekalah','merupakan','meski','meskipun','meyakini','meyakinkan','minta','mirip','misal','misalkan','misalnya','mula','mulai','mulailah','mulanya','mungkin','mungkinkah','nah','naik','namun','nanti','nantinya','nyaris','nyatanya','oleh','olehnya','pada','padahal','padanya','pak','paling','panjang','pantas','para','pasti','pastilah','penting','pentingnya','per','percuma','perlu','perlukah','perlunya','pernah','persoalan','pertama','pertama-tama','pertanyaan','pertanyakan','pihak','pihaknya','pukul','pula','pun','punya','rasa','rasanya','rata','rupanya','saat','saatnya','saja','sajalah','saling','sama','sama-sama','sambil','sampai','sampai-sampai','sampaikan','sana','sangat','sangatlah','satu','saya','sayalah','se','sebab','sebabnya','sebagai','sebagaimana','sebagainya','sebagian','sebaik','sebaik-baiknya','sebaiknya','sebaliknya','sebanyak','sebegini','sebegitu','sebelum','sebelumnya','sebenarnya','seberapa','sebesar','sebetulnya','sebisanya','sebuah','sebut','sebutlah','sebutnya','secara','secukupnya','sedang','sedangkan','sedemikian','sedikit','sedikitnya','seenaknya','segala','segalanya','segera','seharusnya','sehingga','seingat','sejak','sejauh','sejenak','sejumlah','sekadar','sekadarnya','sekali','sekali-kali','sekalian','sekaligus','sekalipun','sekarang','sekarang','sekecil','seketika','sekiranya','sekitar','sekitarnya','sekurang-kurangnya','sekurangnya','sela','selain','selaku','selalu','selama','selama-lamanya','selamanya','selanjutnya','seluruh','seluruhnya','semacam','semakin','semampu','semampunya','semasa','semasih','semata','semata-mata','semaunya','sementara','semisal','semisalnya','sempat','semua','semuanya','semula','sendiri','sendirian','sendirinya','seolah','seolah-olah','seorang','sepanjang','sepantasnya','sepantasnyalah','seperlunya','seperti','sepertinya','sepihak','sering','seringnya','serta','serupa','sesaat','sesama','sesampai','sesegera','sesekali','seseorang','sesuatu','sesuatunya','sesudah','sesudahnya','setelah','setempat','setengah','seterusnya','setiap','setiba','setibanya','setidak-tidaknya','setidaknya','setinggi','seusai','sewaktu','siap','siapa','siapakah','siapapun','sini','sinilah','soal','soalnya','suatu','sudah','sudahkah','sudahlah','supaya','tadi','tadinya','tahu','tahun','tak','tambah','tambahnya','tampak','tampaknya','tandas','tandasnya','tanpa','tanya','tanyakan','tanyanya','tapi','tegas','tegasnya','telah','tempat','tengah','tentang','tentu','tentulah','tentunya','tepat','terakhir','terasa','terbanyak','terdahulu','terdapat','terdiri','terhadap','terhadapnya','teringat','teringat-ingat','terjadi','terjadilah','terjadinya','terkira','terlalu','terlebih','terlihat','termasuk','ternyata','tersampaikan','tersebut','tersebutlah','tertentu','tertuju','terus','terutama','tetap','tetapi','tiap','tiba','tiba-tiba','tidak','tidakkah','tidaklah','tiga','tinggi','toh','tunjuk','turut','tutur','tuturnya','ucap','ucapnya','ujar','ujarnya','umum','umumnya','ungkap','ungkapnya','untuk','usah','usai','waduh','wah','wahai','waktu','waktunya','walau','walaupun','wong','yaitu','yakin','yakni','yang']

sw=[]

for conten in PK:
    kata= list(filter(lambda x: x not in stopword, conten))
    sw.append(kata)

df['stopword'] = sw

Setelah menghilangkan kata-kata yang tidak penting akan tersisa kata-kata yang penting saja pada data. lalu kita assign algoritma TF dan TF-IDF yang akan digunakan dalam pembuatan keywords

In [9]:
#Perhitungan TF dan TF-IDF

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

## Pembuatan Keywords dengan Term Frequency

Dalam task ini saya mencoba menggunakan perhitungan term frequency untuk menentukan keywords pada content berita, dimana term frequency akan menampilkan hasil perhitungan dari total kata muncul dibagi dengan total semua kata pada suatu dokumen.

In [10]:
#Membuat keyword dengan perhitungan tf(term frequency)

document1 = tb(' '.join(df['stopword'][0]))
document2 = tb(' '.join(df['stopword'][1]))
document3 = tb(' '.join(df['stopword'][2]))
document4 = tb(' '.join(df['stopword'][3]))

bloblist = [document1,document2,document3,document4]
for i, blob in enumerate(bloblist):
    print("Keywords pada dokumen {}".format(i + 1))
    scores = {word: tf(word, blob) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:10]:
        print("Word: {}, TF: {}".format(word, round(score, 5)))

Keywords pada dokumen 1
Word: Kepolisian, TF: 0.04587
Word: kepolisian, TF: 0.04587
Word: orang, TF: 0.0367
Word: London, TF: 0.0367
Word: kereta, TF: 0.02752
Word: Inggris, TF: 0.02752
Word: bom, TF: 0.02752
Word: ratusan, TF: 0.01835
Word: stasiun, TF: 0.01835
Word: 22, TF: 0.01835
Keywords pada dokumen 2
Word: Tol, TF: 0.05042
Word: tol, TF: 0.05042
Word: Jalan, TF: 0.03782
Word: elektronik, TF: 0.03782
Word: jalan, TF: 0.03782
Word: uang, TF: 0.03361
Word: jasa, TF: 0.02101
Word: transaksi, TF: 0.02101
Word: Jasa, TF: 0.02101
Word: maksimal, TF: 0.01681
Keywords pada dokumen 3
Word: tagar, TF: 0.10638
Word: Twitter, TF: 0.04255
Word: sosial, TF: 0.03191
Word: media, TF: 0.03191
Word: 10, TF: 0.03191
Word: platform, TF: 0.02128
Word: pasca, TF: 0.02128
Word: topik, TF: 0.02128
Word: 23/8, TF: 0.01064
Word: penggunanya, TF: 0.01064
Keywords pada dokumen 4
Word: Cadangan, TF: 0.07981
Word: cadangan, TF: 0.07981
Word: BBM, TF: 0.07512
Word: nasional, TF: 0.02817
Word: Indonesia, TF: 0.

## Pembuatan Keywords dengan Term Frequency-Inverse Document Frequency

Dalam task ini saya juga mencoba menggunakan perhitungan tf-idf untuk menentukan keywords pada content berita, dimana tf-idf akan menampilkan hasil perhitungan dari total kata muncul dibagi dengan total semua kata pada suatu dokumen yang dibandingkan dengan kemunculan kata di dokumen lain.

In [11]:
#Membuat keyword dengan perhitungan tfidf(term frequency-inverse document frequency)

document1 = tb(' '.join(df['stopword'][0]))
document2 = tb(' '.join(df['stopword'][1]))
document3 = tb(' '.join(df['stopword'][2]))
document4 = tb(' '.join(df['stopword'][3]))

bloblist = [document1,document2,document3,document4]
for i, blob in enumerate(bloblist):
    print("Keywords pada dokumen {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:10]:
        print("Word: {}, TF-IDF: {}".format(word, round(score, 5)))

Keywords pada dokumen 1
Word: Kepolisian, TF-IDF: 0.0318
Word: kepolisian, TF-IDF: 0.0318
Word: London, TF-IDF: 0.02544
Word: kereta, TF-IDF: 0.01908
Word: Inggris, TF-IDF: 0.01908
Word: bom, TF-IDF: 0.01908
Word: ratusan, TF-IDF: 0.01272
Word: stasiun, TF-IDF: 0.01272
Word: Sedikitnya, TF-IDF: 0.01272
Word: Peristiwa, TF-IDF: 0.01272
Keywords pada dokumen 2
Word: Tol, TF-IDF: 0.03495
Word: tol, TF-IDF: 0.03495
Word: Jalan, TF-IDF: 0.02621
Word: elektronik, TF-IDF: 0.02621
Word: jalan, TF-IDF: 0.02621
Word: uang, TF-IDF: 0.0233
Word: jasa, TF-IDF: 0.01456
Word: transaksi, TF-IDF: 0.01456
Word: Jasa, TF-IDF: 0.01456
Word: maksimal, TF-IDF: 0.01165
Keywords pada dokumen 3
Word: tagar, TF-IDF: 0.07374
Word: Twitter, TF-IDF: 0.0295
Word: sosial, TF-IDF: 0.02212
Word: media, TF-IDF: 0.02212
Word: platform, TF-IDF: 0.01475
Word: pasca, TF-IDF: 0.01475
Word: topik, TF-IDF: 0.01475
Word: 10, TF-IDF: 0.00918
Word: 23/8, TF-IDF: 0.00737
Word: penggunanya, TF-IDF: 0.00737
Keywords pada dokumen 4


## Evaluasi

Hasil dari keyword tersebut berdasarkan hasil dari perhitungan dari Term Frequncy didapatkan dari perhitungan pada dokumen itu sendiri, dan semakin banyak kata yang muncul akan mempunyai nilai yang tinggi.
Untuk TF-IDF keakuratanya masih belum bisa di pastikan, dan TF-IDF akan bekerja optimal bila data yang disediakan proporsional serta dengan topik yang berimbang.

## Future Works
- Mencoba metode lain apakah bisa lebih bagus dengan metode tersebut.
- Mencoba menambah proses yang diperlukan pada preprocessing data seperti stemming, dan lain sebagainya